In [9]:
#pastas colocadas e que funcionam:
#seguenotasfiscais2quinzenadedezembropnz funcionando<<
#n_f52transf_jpajabjpa (deve funcionar mas só 1 file)
#n_fs1718jaymejosedelimajunio (deve funcionar mas só 2 file)
#nfs_henrique_roberval_josrogrio funcionando<<
#nfs_jarbiana não_funcionando <<<<<<<<MAS TA LENDO>>>>>>>>
#nfsref2quinzenadezembro2023 tem que ser feito com o leitor de imagem
#ren_fs535455e56edvaldodesouzasilvajunior funcionando <<<

essencialmente só colocar o dir da pasta na função que ele faz um csv com a informação

In [7]:
def leitor(dir):
    import os
    import csv  
    import pdfplumber
    import pandas as pd
    
    csv_file_path = dir + '/pdf_analise.csv'
    colunas = ('chave','cnpj_prestador','cnpj_tomador','valor_servico','numero_nota','data','email_tomador','email_prestador','empresa_tomador','empresa_prestador','codigo_tributacao','IRRF,CP,CSLL')            
    with open(csv_file_path,'w',newline='') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(colunas)
    try:   
        fileExt = r".pdf"
        files = os.listdir(dir)
        pdf_files = [arq for arq in files if arq.lower().endswith('.pdf')]
        for pdf_file in pdf_files:
            with pdfplumber.open(f"{dir}/{pdf_file}") as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text()
                    a = text
                    lines = a.split('\n')
                    cda=False
                    tds=False
                    vds=False
                    nfs=False
                    ne=False
                    em=0
                    cdt=False
                    counter=0
                    p_irff = False
                    for line in lines:
                        if p_irff == True:
                            irrf = line.split(' ')[0]
                            p_irff = False
                        if cdt==True:
                            counter=counter+1
                            if counter==1:
                                codigo_tributacao=line.split(' ')[0].split('-')[0]
                                desc1=line.split(' ')[1]
                                
                            if counter==2:
                                desc2=line.split(' ')[0]
                                desc=desc1+desc2
                                cdt=False
                            cdt=cdt+1
                            
                        if ne==True:
                            if em == 0:
                                email_prestador=line.split(' ')[1]
                                empresa_tomador = line.split('-')[0]
                                em =1
                            email_tomador=line.split(' ')[1]
                            empresa_prestador = line.split(' ')[0]
                            ne=False
                        if nfs == True:
                            numero_nota= line.split(' ')[0]
                            data = line.split(' ')[1]
                            nfs=False
                        if vds == True:
                            valor_servico=line.split(' ')[0]
                            vds=False
                        if tds== True:
                            cnpj_tomador = line.split(' ')[0]
                            tds = False
                        if cda == True:
                            chave = line.replace(' ','')[0:44]
                            cda=False
                        if 'ChavedeAcessodaNFS' in line:
                            cda=True
                        
                        if 'PrestadordoServiço' in line:
                            cnpj_prestador = line.split(' ')[1]
                        if 'TOMADORDOSERVIÇO' in line:
                            tds=True
                            
                        if 'ValordoServiço' in line:
                            vds=True
                            
                        if 'NúmerodaNFS' in line:
                            nfs=True
                            
                        if 'Nome/NomeEmpresarial' in line:
                            ne=True
                            
                        if 'CódigodeTributaçãoNacional' in line:
                            cdt=True
                            
                        if 'IRRF,CP,CSLL' in line:
                            p_irff=True
                    with open(csv_file_path, mode='a', encoding= 'utf8',newline='') as resultado_pesquisa:
                        writer = csv.writer(resultado_pesquisa)
                        linhas = (chave,cnpj_prestador,cnpj_tomador,valor_servico,numero_nota,data,email_tomador,email_prestador,empresa_tomador,empresa_prestador,codigo_tributacao,irrf)
                        writer.writerow(linhas)
    except:
        print('não foi possível ler todos os dados')
            
                #(r"C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz/LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf")
            


teste

In [5]:
import pdfplumber
cont = 0
with pdfplumber.open(r"C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz/LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf") as pdf:
            text = ""
            cnpj_prestador = 0
            naprox = False
            csll_prox = False
            ded_prox = False
            for page in pdf.pages:
                text += page.extract_text()
                a = text
                lines = a.split('\n')
                for line in lines: 
                    if 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFSe' == lines[0]:
                        if ded_prox == True:
                            VALOR_SERVIÇO = line.split(' ')[0]
                            DEDUÇÕES = line.split(' ')[1]
                            DESCONTO_INCONDICIONAL = line.split(' ')[2]
                            BASE_CÁLCULO = line.split(' ')[3]
                            
                            ISS = line.split(' ')[5]
                            ded_prox = False
                        if csll_prox == True:
                            INSS = line.split(' ')[0]
                            IR = line.split(' ')[1]
                            CSLL = line.split(' ')[2]
                            COFINS = line.split(' ')[3]
                            PIS = line.split(' ')[4]
                            csll_prox = False
                        if naprox == True:
                            nota = line.split(' ')[0]
                            naprox=False
                        if 'CPF/CNPJ:' in line:
                            cnpj_tomador = line.split(' ')[-1]
                            if cnpj_prestador == 0:
                                cnpj_prestador = line.split(' ')[-1]
                        if 'Emitido em' in line:
                            data = line.split(' ')[2] 
                        if 'Exigível Tributacao Normal' in line:
                            naprox=True
                        if 'CSLL (R$)' in line:
                            csll_prox = True
                            
                        if 'DEDUÇÕES' in line:
                            ded_prox = True
                        print(line,cont)
                print(cnpj_prestador,cnpj_tomador,data,nota,VALOR_SERVIÇO ,DEDUÇÕES ,DESCONTO_INCONDICIONAL,ISS)
            

NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFSe 0
MUNICIPIO DE PETROLINA 0
Codigo de Verificação para Autenticação: 22bd2510d 0
Endereço: Petrolina, Pernambuco, PE, 56302-000 0
CNPJ: 10.358.190/0001-77, E-mail: GABINETEPREFEITURAPETROLINA@HOTMAIL.COM 0
Emitido em 02/01/2024 08:38:20 0
Data Fato Gerador Exigibilidade de ISS Regime Tributário Número RPS Nº da Nota Fiscal 0
02/01/2024 Exigível Tributacao Normal 0
119 0
Tipo de Recolhimento Simples Local de Prestação Local de Recolhimento 0
Não Retido Optante 2611101 - Petrolina - PE 2611101 - Petrolina - PE 0
PRESTADOR 0
Razão Social: LAZARO JOEDSON MOURA SILVA 0
Nome Fantasia: MILHAO DO VALE 0
Endereço: Rua NUCLEO RURAL PSNC - N7, 115, LETRA D - ZONA RURAL 0
Petrolina - PE - CEP: 56302-970 0
E-mail: contabilidadedovale@hotmail.com - Fone: - Site: ........ 0
Inscrição Estadual: ........ - Inscrição Municipal: 9632190 - CPF/CNPJ: 24.254.072/0001-23 0
TOMADOR 0
Razão Social: PROGRESSO LOGISTICA E TRANSPORTE LTDA 0
Endereço: Rua SETE DE SETEMBRO, 

In [8]:
leitor('C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz')

In [3]:
leitor(r'C:\Users\jvict\Documents\seven\seven\n_f52transf_jpajabjpa')

In [4]:

leitor(r'C:/Users/jvict/Documents/seven/seven/nfs_henrique_roberval_josrogrio')


In [32]:
cont = 0
with pdfplumber.open(r"C:\Users\jvict\Documents\seven\seven\ren_fs535455e56edvaldodesouzasilvajunior\NF 53 HZD6008 COLETA FRIOVIX 18 A 29.pdf") as pdf:
            text = ""
            cnpj_prestador = 0
            naprox = False
            csll_prox = False
            for page in pdf.pages:
                text += page.extract_text()
                a = text
                lines = a.split('\n')
                for line in lines: 
                    print(line)

JOÃOPESSOA
DANFSev1.0 (83)3213-7403
DocumentoAuxiliardaNFS-e SEREM.DIFISDIR@JOAOPESSOA.PB.GOV.BR
ChavedeAcessodaNFS-e
25075072223159548000184000000000005324017802126193
NúmerodaNFS-e CompetênciadaNFS-e DataeHoradaemissãodaNFS-e
53 02/01/2024 02/01/202411:04:06
NúmerodaDPS SériedaDPS DataeHoradaemissãodaDPS
59 900 02/01/202411:04:06 AautenticidadedestaNFS-epodeserverificada
pelaleituradestecódigoQRoupelaconsultada
chavedeacessonoportalnacionaldaNFS-e
EMITENTEDANFS-e CNPJ/CPF/NIF InscriçãoMunicipal Telefone
PrestadordoServiço 23.159.548/0001-84 - (83)9985-8336
Nome/NomeEmpresarial E-mail
EDVALDODESOUZASILVAJUNIOR07562256470 -
Endereço Município CEP
ALINASARAIVADEOLIVEIRA,SN JoãoPessoa-PB 58067-256
SimplesNacionalnaDatadeCompetência RegimedeApuraçãoTributáriapeloSN
Optante-MicroempreendedorIndividual(MEI) -
TOMADORDOSERVIÇO CNPJ/CPF/NIF InscriçãoMunicipal Telefone
10.580.938/0010-72 - -
Nome/NomeEmpresarial E-mail
PROGRESSOLOGISTICAETRANSPORTELTDA EDUARDO@HECONTABIL.COM.BR
Endereço Municí

In [35]:
leitor(r"C:\Users\jvict\Documents\seven\seven\ren_fs535455e56edvaldodesouzasilvajunior")